In [1]:
import tensorflow as tf
from keras.layers import Conv2D, BatchNormalization, ReLU, DepthwiseConv2D, Concatenate, GlobalAveragePooling2D, Dense
from keras.models import Model

In [3]:
def ghost_module(inputs, filters, kernel_size=(3, 3), ratio=2):
    init_channels = int(filters / ratio)
    new_channels = int(filters - init_channels)

    # Primary Convolution
    x = Conv2D(init_channels, kernel_size, strides=(1, 1), padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Cheap Operation
    cheap = DepthwiseConv2D(kernel_size, strides=(1, 1), padding='same', use_bias=False)(x)
    cheap = BatchNormalization()(cheap)
    cheap = ReLU()(cheap)

    # Concatenating the outputs
    cheap = Conv2D(new_channels, (1, 1), strides=(1, 1), padding='same', use_bias=False)(cheap)
    cheap = BatchNormalization()(cheap)
    cheap = ReLU()(cheap)

    x = Concatenate()([x, cheap])
    return x


In [4]:
def GhostNet(input_shape=(224, 224, 3), classes=2):
    inputs = tf.keras.Input(shape=input_shape)

    # Initial Layer
    x = Conv2D(16, (3, 3), strides=(2, 2), padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Example of applying Ghost Modules
    x = ghost_module(x, 16)
    # ... Continue building the network with more Ghost modules and other necessary layers

    # Final Layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs, outputs=x)
    return model


In [5]:
model = GhostNet(input_shape=(224, 224, 3), classes=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with your data
# history = model.fit(train_generator, epochs=10, validation_data=validation_generator)


In [6]:
from keras.preprocessing.image import ImageDataGenerator

# Define paths to your dataset
train_data_dir = './train/'
test_data_dir = './test/'
val_data_dir = './val/'

# Set parameters
img_width, img_height = 224, 224
batch_size = 64
epochs = 15

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # assuming binary classification (drowsy and non-drowsy)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2149 images belonging to 2 classes.
Found 272 images belonging to 2 classes.
Found 268 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()